In [8]:
import xml.dom.minidom as Dom
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn import linear_model
import xml.etree.ElementTree as et
import numpy as np

def get_dic(string):
    #get dict from xml
    train_labels = et.parse(string)
    root = train_labels.getroot()
    dic = {}
    for child in root:
        key = child.attrib['source']
        sub_dic = {}
        for sub_child in child:
            sub_key =  sub_child.attrib['name']
            value = []
            for sub_sub_child in sub_child:
                value.append(sub_sub_child.attrib['id']+','+sub_sub_child.attrib['judgment'])
            sub_dic[sub_key]= value
        dic[key] = sub_dic    
    return dic

f = open('data/obesity_doc_200_java.vec', 'r')
content = f.read()
f.close()

lines = content.split('\n')
embedding_vectors = []
for line in lines:
    values = line.split(' ')
    vector = values[1:]
    embedding_vectors.append(vector)
print(embedding_vectors[0])
print(len(embedding_vectors))
print(len(embedding_vectors[0]))

train_dic = get_dic('data/Obesity_data/train_groundtruth.xml')
#print(train_dic) 
test_dic = get_dic('data/Obesity_data/test_groundtruth.xml')
#print(test_dic) 


doc = Dom.Document() 
root_node = doc.createElement("diseaseset")
doc.appendChild(root_node) 

for key in train_dic:
    train_sub_dic = train_dic[key]
    test_sub_dic = test_dic[key]
    source_node = doc.createElement("diseases")
    source_node.setAttribute("source", key)
    for sub_key in train_sub_dic:
        disease_node = doc.createElement("disease")
        disease_node.setAttribute("name", sub_key)
        train_data_X = []
        train_data_Y = [] 
        train_docs = train_sub_dic[sub_key]
        print(len(train_docs))
        for train_doc in train_docs:
            temp = train_doc.split(',')
            #print(key,sub_key,temp)
            train_data_X.append(embedding_vectors[int(temp[0])-1])
            train_data_Y.append(temp[1])
        clf = svm.SVC(decision_function_shape='ovr',class_weight="balanced",kernel='linear')
        #clf = GaussianNB()
        #clf = linear_model.LogisticRegression(C=1e5, class_weight = 'balanced')
        train_data_X = np.array(train_data_X)
        train_data_Y = np.array(train_data_Y)
        clf.fit(train_data_X, train_data_Y)
    
        test_docs = test_sub_dic[sub_key]
        test_data_X = []
        test_data_Y = [] 
        print(len(test_docs))
    
        for test_doc in test_docs:
            temp = test_doc.split(',')
            test_data_X.append(embedding_vectors[int(temp[0])-1])
            test_data_Y.append(temp[1])
        
        predict_y = clf.predict(test_data_X)
        print(predict_y)
    
        correct_count = 0
        for i in range(len(test_data_Y)):
            if predict_y[i] == test_data_Y[i]:
                correct_count += 1
            doc_node = doc.createElement("doc")
            doc_node.setAttribute("id", test_docs[i].split(',')[0])
            doc_node.setAttribute("judgment", predict_y[i])
            disease_node.appendChild(doc_node) 
        accuracy = correct_count * 1.0/ len(test_data_Y)
        print(key + ','+ sub_key + ' : Accuaracy :'+ str(accuracy))
    
        source_node.appendChild(disease_node) 
        root_node.appendChild(source_node)
    
f = open("test_predict_doc2vec_remove_family.xml", "w") 
f.write(doc.toprettyxml(indent = "", newl = "\n", encoding = "utf-8")) 
f.close()

['-0.14198759', '-0.699901', '0.1772508', '-0.3630808', '0.20355467', '0.104961306', '0.11658239', '0.16597617', '0.53036064', '0.92252463', '0.33263433', '0.16159691', '-0.050322134', '-0.64508283', '0.863681', '-0.35583475', '-0.8511803', '0.37400568', '-0.65686744', '0.4834386', '0.27768585', '-0.19071816', '-1.3597044', '1.0795108', '0.18876575', '0.36257288', '0.6002793', '0.23875675', '-0.31708553', '0.21964493', '0.23948154', '0.032630093', '0.59415257', '-0.6514114', '-0.22885042', '0.3664483', '-0.03935495', '0.50272983', '0.48426527', '-0.12181851', '0.30160338', '-0.1641868', '0.5083092', '0.2388725', '-0.32691088', '0.31338376', '-0.23184182', '-0.048911005', '-1.0409696', '-0.27991742', '-0.40818873', '0.42561936', '-0.25460327', '-0.12879936', '0.32009786', '-0.4560489', '0.101752274', '0.294658', '0.27120295', '-0.001067423', '-0.37119234', '-0.76129717', '1.0926484', '-0.3884036', '0.47788116', '0.40397415', '-0.028229248', '-0.43607306', '0.14319518', '-0.018709408', '

723
504
['U' 'U' 'U' 'Q' 'Y' 'U' 'U' 'Y' 'U' 'U' 'Y' 'U' 'U' 'U' 'Y' 'Y' 'U' 'Y'
 'U' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'U' 'Y' 'Y' 'U' 'U' 'Y' 'U' 'Y' 'U' 'U'
 'U' 'U' 'Y' 'Y' 'Y' 'U' 'U' 'U' 'U' 'Y' 'Y' 'U' 'Y' 'U' 'U' 'U' 'U' 'U'
 'U' 'U' 'Y' 'U' 'Y' 'U' 'Y' 'U' 'U' 'Y' 'U' 'U' 'Q' 'Y' 'Y' 'Y' 'Y' 'U'
 'Y' 'U' 'Y' 'Y' 'U' 'U' 'Y' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'Y' 'U' 'U'
 'U' 'U' 'U' 'Y' 'Y' 'Y' 'U' 'U' 'Y' 'U' 'Y' 'Y' 'U' 'U' 'U' 'U' 'U' 'U'
 'U' 'U' 'U' 'U' 'U' 'U' 'Y' 'U' 'U' 'U' 'U' 'U' 'Y' 'Y' 'Y' 'Q' 'U' 'U'
 'Y' 'Y' 'Y' 'Y' 'Y' 'U' 'U' 'Y' 'U' 'U' 'U' 'U' 'U' 'U' 'Y' 'U' 'U' 'U'
 'U' 'Y' 'U' 'U' 'U' 'U' 'U' 'U' 'Y' 'Y' 'U' 'Q' 'U' 'Y' 'U' 'U' 'U' 'Y'
 'U' 'U' 'U' 'Y' 'Y' 'Y' 'U' 'Y' 'Y' 'U' 'U' 'U' 'U' 'Y' 'U' 'U' 'U' 'U'
 'Y' 'Y' 'U' 'Y' 'Y' 'U' 'Y' 'Y' 'Y' 'U' 'U' 'Y' 'U' 'U' 'Y' 'U' 'U' 'Y'
 'U' 'Y' 'Y' 'Y' 'U' 'U' 'U' 'U' 'Y' 'U' 'U' 'Y' 'U' 'U' 'U' 'U' 'U' 'U'
 'U' 'Y' 'U' 'U' 'U' 'Y' 'U' 'U' 'U' 'Y' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'Y'
 'U' 'Y' 'U' 'U' 'U' 'U' 'U' 'Y' 'U' 'Y' 'U

507
['U' 'U' 'Y' 'U' 'U' 'U' 'U' 'Y' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U'
 'U' 'U' 'U' 'Y' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U'
 'U' 'U' 'U' 'U' 'U' 'Y' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'Y' 'U' 'U'
 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'Y' 'U' 'U' 'U' 'U' 'U' 'U'
 'U' 'U' 'U' 'U' 'U' 'U' 'Y' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U'
 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U'
 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U'
 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'Y' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U'
 'U' 'Y' 'U' 'U' 'U' 'U' 'U' 'U' 'Y' 'U' 'U' 'U' 'U' 'U' 'U' 'Y' 'U' 'U'
 'U' 'U' 'U' 'U' 'U' 'U' 'Y' 'U' 'Y' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U'
 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U'
 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U'
 'Y' 'U' 'U' 'U' 'U' 'U' 'U' 'Y' 'U' 'U' 'U' 'U' 'Y' 'U' 'U' 'U' 'U' 'U'
 'Y' 'U' 'U' 'Y' 'U' 'U' 'U' 'U' 'U' 'U' 'U' 'U

493
['Y' 'U' 'Y' 'Y' 'Y' 'Y' 'U' 'U' 'Y' 'U' 'U' 'Y' 'U' 'Y' 'Y' 'Y' 'Y' 'Y'
 'Y' 'Y' 'U' 'Y' 'Y' 'U' 'Y' 'Y' 'Y' 'U' 'Y' 'Y' 'Y' 'U' 'U' 'U' 'U' 'U'
 'U' 'U' 'Y' 'Y' 'U' 'U' 'U' 'Y' 'Y' 'Y' 'U' 'U' 'Y' 'Y' 'Y' 'U' 'U' 'U'
 'U' 'Y' 'Y' 'Y' 'U' 'N' 'Y' 'U' 'U' 'Y' 'Y' 'U' 'Y' 'U' 'U' 'U' 'U' 'Y'
 'U' 'Y' 'N' 'U' 'Y' 'Y' 'Y' 'Y' 'U' 'U' 'U' 'U' 'U' 'Y' 'U' 'U' 'U' 'U'
 'U' 'Y' 'U' 'N' 'U' 'Y' 'U' 'Y' 'U' 'Y' 'U' 'U' 'Y' 'Y' 'U' 'Y' 'U' 'Y'
 'Y' 'Y' 'Y' 'U' 'U' 'U' 'U' 'Y' 'Y' 'U' 'U' 'U' 'U' 'U' 'U' 'Y' 'Y' 'U'
 'Y' 'U' 'U' 'U' 'U' 'U' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'U' 'U' 'U' 'Y' 'Y' 'U'
 'Y' 'U' 'Y' 'U' 'Y' 'U' 'U' 'Y' 'U' 'Y' 'Y' 'Y' 'Y' 'U' 'Y' 'U' 'Y' 'U'
 'Y' 'U' 'U' 'U' 'U' 'Y' 'Y' 'Y' 'U' 'U' 'Y' 'Y' 'U' 'U' 'U' 'Y' 'U' 'Y'
 'U' 'U' 'U' 'Y' 'U' 'U' 'Y' 'Y' 'U' 'U' 'U' 'U' 'Y' 'U' 'U' 'Y' 'U' 'Y'
 'Y' 'Y' 'U' 'U' 'Y' 'Y' 'Y' 'U' 'U' 'Y' 'Y' 'Y' 'U' 'Y' 'U' 'U' 'U' 'Y'
 'Y' 'Y' 'Y' 'Y' 'U' 'U' 'U' 'U' 'U' 'Y' 'Y' 'Y' 'U' 'U' 'U' 'U' 'U' 'U'
 'U' 'U' 'Y' 'U' 'Y' 'Y' 'U' 'U' 'Y' 'U' 'U' 'U

503
['U' 'N' 'U' 'Y' 'U' 'U' 'Y' 'U' 'U' 'Y' 'Y' 'Q' 'Y' 'Y' 'U' 'U' 'Y' 'U'
 'U' 'Y' 'Y' 'U' 'U' 'U' 'U' 'Y' 'Y' 'U' 'Y' 'U' 'U' 'Y' 'U' 'Y' 'Y' 'U'
 'U' 'U' 'Y' 'Y' 'U' 'Y' 'Y' 'U' 'Q' 'N' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y'
 'U' 'U' 'U' 'Y' 'Y' 'Y' 'U' 'Q' 'U' 'Y' 'U' 'Y' 'Y' 'U' 'Y' 'Y' 'Y' 'Y'
 'Y' 'U' 'Y' 'Y' 'Y' 'Q' 'Y' 'U' 'Y' 'U' 'U' 'Y' 'U' 'N' 'Y' 'Y' 'Y' 'U'
 'Y' 'Y' 'Y' 'U' 'Y' 'U' 'Y' 'Y' 'Y' 'U' 'Y' 'Y' 'Y' 'Y' 'Y' 'U' 'Y' 'Y'
 'Y' 'Y' 'U' 'Y' 'Y' 'U' 'U' 'Y' 'Y' 'Y' 'U' 'U' 'Y' 'Y' 'Y' 'U' 'Y' 'Y'
 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'N' 'Y' 'U' 'Y' 'U' 'U' 'Y' 'U' 'Y' 'U' 'Q' 'U'
 'U' 'U' 'Y' 'N' 'U' 'N' 'Y' 'N' 'Y' 'U' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'U'
 'Y' 'U' 'Y' 'Y' 'Y' 'Y' 'Y' 'Q' 'U' 'Y' 'Y' 'Y' 'U' 'N' 'Y' 'U' 'Y' 'Y'
 'N' 'Q' 'U' 'N' 'U' 'Q' 'Y' 'U' 'Y' 'Y' 'Y' 'Y' 'U' 'Y' 'Y' 'Y' 'U' 'U'
 'U' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'U' 'Y' 'Y' 'U' 'Y' 'Y' 'Y' 'Y' 'U'
 'U' 'Y' 'U' 'U' 'U' 'N' 'Y' 'Y' 'N' 'Y' 'U' 'Y' 'Y' 'U' 'Y' 'Y' 'U' 'Y'
 'N' 'Y' 'Y' 'U' 'Y' 'U' 'Y' 'U' 'Y' 'Y' 'U' 'Y

intuitive,GERD : Accuaracy :0.570422535211
710
491
['N' 'N' 'N' 'Y' 'N' 'Y' 'N' 'N' 'N' 'Y' 'N' 'Y' 'N' 'N' 'N' 'N' 'N' 'N'
 'N' 'Y' 'N' 'N' 'Y' 'Y' 'Y' 'N' 'Y' 'N' 'Y' 'Y' 'N' 'N' 'Y' 'N' 'N' 'N'
 'Y' 'N' 'N' 'N' 'Y' 'N' 'N' 'Y' 'N' 'N' 'N' 'Y' 'N' 'Y' 'N' 'Y' 'N' 'Y'
 'Y' 'N' 'Y' 'N' 'N' 'N' 'Y' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'Y' 'N'
 'N' 'Y' 'N' 'N' 'N' 'N' 'N' 'N' 'Y' 'N' 'Y' 'Y' 'Y' 'N' 'N' 'N' 'Y' 'Y'
 'N' 'N' 'N' 'Y' 'N' 'Y' 'N' 'N' 'N' 'Y' 'N' 'Y' 'N' 'Y' 'Y' 'N' 'N' 'N'
 'N' 'N' 'N' 'Y' 'N' 'Y' 'N' 'Y' 'Y' 'N' 'N' 'N' 'N' 'Y' 'N' 'N' 'N' 'N'
 'N' 'N' 'N' 'Y' 'Y' 'Y' 'N' 'N' 'N' 'N' 'N' 'Y' 'N' 'Y' 'N' 'N' 'Y' 'Y'
 'Y' 'Y' 'Y' 'Y' 'N' 'N' 'N' 'N' 'N' 'N' 'Y' 'Y' 'Y' 'N' 'Y' 'N' 'N' 'Y'
 'N' 'N' 'N' 'Y' 'N' 'Y' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'Y' 'Y'
 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'Y' 'N' 'Y' 'Y' 'Y' 'Y' 'Y' 'N' 'Y'
 'Y' 'N' 'Y' 'Y' 'N' 'N' 'N' 'N' 'Y' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N'
 'N' 'N' 'Y' 'N' 'Y' 'Y' 'Y' 'N' 'Y' 'Y' 'N' 'Y' 'N' 'Y' 'N' 'N' 'N' 'N'


672
462
['N' 'N' 'N' 'Y' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'Y' 'N' 'Y' 'Y' 'N' 'N'
 'Y' 'Y' 'Y' 'Y' 'Y' 'N' 'Y' 'N' 'Y' 'N' 'Y' 'Y' 'Y' 'N' 'N' 'N' 'N' 'N'
 'N' 'N' 'Y' 'N' 'Y' 'N' 'Y' 'N' 'Y' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N'
 'Y' 'N' 'Y' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'Y' 'N' 'N' 'N' 'Y' 'N' 'Y' 'N'
 'N' 'Y' 'Y' 'N' 'Y' 'N' 'N' 'N' 'N' 'N' 'N' 'Y' 'N' 'Y' 'N' 'N' 'N' 'N'
 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'Y' 'N' 'N'
 'N' 'N' 'N' 'Y' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'Y' 'Y' 'Y' 'N' 'N' 'Y' 'N'
 'Y' 'N' 'Y' 'Y' 'Y' 'Y' 'N' 'N' 'Y' 'N' 'Y' 'N' 'Y' 'Y' 'N' 'N' 'N' 'Y'
 'N' 'N' 'Y' 'Y' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'Y' 'N' 'N' 'N' 'N' 'N' 'N'
 'N' 'Y' 'N' 'N' 'N' 'N' 'Y' 'N' 'Y' 'N' 'Y' 'N' 'Y' 'N' 'N' 'N' 'N' 'Y'
 'N' 'Y' 'N' 'Y' 'N' 'N' 'Y' 'Y' 'N' 'N' 'Y' 'N' 'N' 'N' 'Y' 'N' 'N' 'N'
 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'Y' 'N' 'N' 'Y' 'Y' 'N' 'N' 'Y' 'N'
 'Y' 'N' 'N' 'N' 'N' 'N' 'Y' 'N' 'N' 'N' 'Y' 'N' 'Y' 'N' 'Y' 'N' 'N' 'Y'
 'N' 'Y' 'Y' 'N' 'N' 'N' 'N' 'N' 'Y' 'Y' 'N

697
477
['N' 'N' 'N' 'Y' 'Y' 'Y' 'N' 'Y' 'Y' 'N' 'Y' 'Y' 'N' 'N' 'N' 'Y' 'N' 'N'
 'N' 'Y' 'N' 'Y' 'Y' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N'
 'Y' 'N' 'N' 'Y' 'N' 'N' 'N' 'Y' 'N' 'Y' 'Y' 'Y' 'N' 'N' 'N' 'N' 'N' 'N'
 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'Y' 'N' 'Y' 'N' 'N' 'N' 'Y' 'N' 'N'
 'Y' 'N' 'Y' 'Y' 'N' 'Y' 'Y' 'Y' 'N' 'N' 'N' 'N' 'Y' 'N' 'N' 'Y' 'N' 'N'
 'N' 'N' 'Y' 'N' 'N' 'N' 'Y' 'N' 'N' 'N' 'N' 'N' 'Y' 'Y' 'Y' 'Y' 'Y' 'N'
 'Y' 'N' 'N' 'N' 'N' 'Y' 'N' 'N' 'N' 'Y' 'N' 'N' 'N' 'N' 'Y' 'Y' 'N' 'N'
 'N' 'N' 'N' 'N' 'N' 'Y' 'Y' 'N' 'N' 'N' 'N' 'N' 'Y' 'N' 'N' 'N' 'N' 'N'
 'N' 'N' 'Y' 'N' 'Y' 'Y' 'N' 'N' 'N' 'Y' 'Y' 'N' 'Y' 'Y' 'Y' 'N' 'N' 'N'
 'Y' 'N' 'N' 'N' 'N' 'Y' 'N' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'N' 'Y' 'N' 'N' 'N'
 'N' 'Y' 'N' 'N' 'N' 'Y' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'Y' 'N' 'N' 'N'
 'N' 'N' 'N' 'Y' 'N' 'Y' 'N' 'Y' 'N' 'N' 'N' 'Y' 'Y' 'N' 'N' 'N' 'N' 'N'
 'Y' 'N' 'Y' 'N' 'N' 'N' 'Y' 'N' 'N' 'N' 'N' 'N' 'Y' 'Y' 'N' 'Y' 'N' 'N'
 'Y' 'Y' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N